## Loading modules and the dataset

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
import xgboost
import bs4
import re

In [2]:
# Dataset
original_df = pd.read_csv("data/train.csv", delimiter=",")

## Linking the questions labels to the questions title
### Using the HTML file, we can access to labels questions which gathered additional information to know how to use these data

In [44]:
html_file = "data/USCODE22_LLCP_102523.HTML"
out_csv = "data/labels_questions.csv"

# Reading HTML 
with open(html_file, encoding="latin-1") as f:
    html = f.read()

soup = bs4.BeautifulSoup(html, "html.parser")

# Extracting blocks of questions
questions = []

for td in soup.find_all("td", class_="l m linecontent"):
    text = td.get_text(separator=" ", strip=True)
    text = text.replace("\xa0", " ")  
    text = re.sub(r"\s+", " ", text)

    # Extracting the different information
    label_match = re.search(r"Label:\s(.*?)\s+(?:Section Name:|Core Section Name:)", text, re.IGNORECASE)
    section_name_match = re.search(r"(?:Section Name:|Core Section Name:)\s(.*?)\s+(?:Section Number:|Core Section Number:|Module Number:)", text, re.IGNORECASE)
    section_number_match = re.search(r"(?:Section Number:|Core Section Number:|Module Number:)\s*([0-9A-Za-z]+)", text, re.IGNORECASE)
    sas_match = re.search(r"SAS Variable Name:\s*([A-Za-z0-9_]+)", text, re.IGNORECASE)

    questions.append({
        "Label": label_match.group(1).strip() if label_match else "",
        "Section Name": section_name_match.group(1).strip() if section_name_match else "",
        "Section Number": section_number_match.group(1).strip() if section_number_match else "", 
        "SAS_Variable_Name": sas_match.group(1).strip() if sas_match else ""
        })

# Conversion to df
labels_questions_df = pd.DataFrame(questions)

# Saving labels df
labels_questions_df.to_csv(out_csv, index=False, encoding="utf-8")

In [ ]:
# Which are the different sections
labels_questions_df["Section Name"].unique()

array(['Record Identification', 'Land Line Introduction',
       'Cell Phone Introduction', 'Respondent Sex', 'Health Status',
       'Healthy Days', 'Health Care Access', 'Exercise',
       'Inadequate Sleep', 'Oral Health', 'Chronic Health Conditions',
       'Demographics', 'Disability',
       'Breast and Cervical Cancer Screening',
       'Colorectal Cancer Screening', 'Tobacco Use',
       'Lung Cancer Screening', 'Alcohol Consumption', 'Immunization',
       'HIV/AIDS', 'Long-term COVID Effects', 'Pre-Diabetes', 'Diabetes',
       'ME/CFS', 'Place of Flu Vaccination', 'HPV Vaccination',
       'Shingles Vaccination', 'COVID Vaccination', 'Respiratory Health',
       'Cancer Survivorship: Type of Cancer',
       'Cancer Survivorship: Course of Treatment',
       'Cancer Survivorship: Pain Management',
       'Prostate Cancer Screening', 'Cognitive Decline', 'Caregiver',
       'Adverse Childhood Experiences',
       'Social Determinants and Health Equity', 'Marijuana Use',
      

In [ ]:
# Which are the different sections
len(labels_questions_df["Section Name"].unique())

56

## Cleaning the dataset
### Removing full NaN columns

In [11]:
# Checking for duplicates 
dup_counts = df['Label'].value_counts()
duplicates = dup_counts[dup_counts > 1]

print(f"Unique labels: {dup_counts.shape[0]}")
print(f"Duplicate labels: {duplicates.shape[0]}")

if duplicates.empty:
    print("No duplicates")
else:
    print("\nDuplicates & Occurrences:")
    print(duplicates)

    # Corresponding line
    dup_rows = df[df['Label'].isin(duplicates.index)].sort_values('Label').reset_index(drop=True)
    print("\nLines Duplicates:")
    print(dup_rows)

Unique labels: 317
Duplicate labels: 5

Duplicates & Occurrences:
Label
Are you male or female?              4
Do you live in college housing?      2
Number of Adults in Household        2
Are you 18 years of age or older?    2
Sexual orientation                   2
Name: count, dtype: int64

Lines Duplicates:
                                Label SAS_Variable_Name
0   Are you 18 years of age or older?           LADULT1
1   Are you 18 years of age or older?           CADULT1
2             Are you male or female?          COLGSEX1
3             Are you male or female?          LANDSEX1
4             Are you male or female?          CELLSEX1
5             Are you male or female?          BIRTHSEX
6     Do you live in college housing?          COLGHOUS
7     Do you live in college housing?          CCLGHOUS
8       Number of Adults in Household          NUMADULT
9       Number of Adults in Household           HHADULT
10                 Sexual orientation            SOMALE
11              

In [20]:
# Merging Similar Label questions : Are you 18 years of age or older? 
complementary_check = ((original_df["LADULT1"].notna() & original_df["CADULT1"].isna()) |
                       (original_df["LADULT1"].isna() & original_df["CADULT1"].notna()))
print(complementary_check.describe())

merged_df = original_df["LADULT1"].fillna(0) + original_df["CADULT1"].fillna(0)


count     225000
unique         1
top         True
freq      225000
dtype: object


In [21]:
# Merging Similar Label questions : Are you 18 years of age or older? 
complementary_check = ((original_df["COLGHOUS"].notna() & original_df["CCLGHOUS"].isna()) |
                       (original_df["COLGHOUS"].isna() & original_df["CCLGHOUS"].notna()))
print(complementary_check.describe())

merged_df = original_df["COLGHOUS"].fillna(0) + original_df["CCLGHOUS"].fillna(0)


count     225000
unique         2
top        False
freq      224249
dtype: object


In [23]:
# Merging Similar Label questions : Are you 18 years of age or older? 
complementary_check = ((original_df["NUMADULT"].notna() & original_df["HHADULT"].isna()) |
                       (original_df["NUMADULT"].isna() & original_df["HHADULT"].notna()))
print(complementary_check.describe())

merged_df = original_df["NUMADULT"].fillna(0) + original_df["HHADULT"].fillna(0)
print(merged_df)


count     225000
unique         2
top         True
freq      224983
dtype: object
0         1.0
1         2.0
2         2.0
3         2.0
4         4.0
         ... 
224995    3.0
224996    1.0
224997    2.0
224998    1.0
224999    2.0
Length: 225000, dtype: float64


## 